## YELL SCRAPPING

> ### Inputs Cell.

In [1]:
search_subject = "architects"
search_location = 'london'# make a check that search makes the 
no_of_results = 50

> ### Dependecies Cell.

In [2]:
url ="https://www.yell.com/?utm_source=byc&utm_medium=referral&utm_campaign=top-black-bar" 
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Firefox
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.events import EventFiringWebDriver, AbstractEventListener
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
Firefox_options = Options()
#Firefox_options.add_argument("--disable-extensions")
# Firefox_options.add_argument("--disable-gpu")
# Firefox_options.add_argument("--headless")
Firefox_options.add_argument("--start-maximized")
# Firefox_options.headless = True # also works
import time
from time import sleep
import pandas as pd
import numpy as np



> ### Scraper Code Cell.

In [3]:
start_time = time.time()
# driver = webdriver.Firefox()
driver = webdriver.Firefox(options=Firefox_options)

actions = ActionChains(driver)
driver.get(url)

search_bar = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'search_keyword'))
)

search_bar.click()
search_bar.send_keys(search_subject)#, Keys.ENTER


location = driver.find_element(By.ID, 'search_location')
search_location_0 = location.get_attribute('value')
location.click()
location.clear()
location.send_keys(search_location)
# location.send_keys(Keys.ENTER)
driver.switch_to.default_content()
print(search_location)
find = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Search now"]')#(By.XPATH, '/html/body/main/div/div/div/div/div/section/div/div/div/form/fieldset/div[1]/div[3]/button')
ActionChains(driver).move_to_element(find).perform()
try:
    sleep(3)
    accept_cookies = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div[2]/button/strong')
    accept_cookies.click()
    driver.switch_to.default_content()
except:
    pass
find.click()
## primary search page done >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
sleep(5)

try:
    results_available_E = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'nav[aria-label="Pagination"]'))
    )# returns the number of resluts available for a given search
    results_available = results_available_E.find_elements(By.CSS_SELECTOR, 'a[class="btn btn-grey"]')[-1].text
except:
    results_available = 1
try:
    pages_to_scrape = no_of_results / 25    #reset the results to be scraped to the available resultss 
    if pages_to_scrape > results_available:  #if it's more than the available results
        no_of_results = results_available * 25
except:
    pass

page = 1
print(results_available, "Pages available.\n")

try:
    accept_cookies = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div[2]/button/strong')
    accept_cookies.click()
    driver.switch_to.default_content()
except:
    pass

result_list =[]

while no_of_results > len(result_list):
   
    if page == 1 :
        try:
            accept_cookies = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[2]/div/div[2]/button/strong')
            accept_cookies.click()
            driver.switch_to.default_content()
        except:
            pass
        
    sleep(5)    
    primary_info_E_0 = driver.find_element(By.ID, "rightNav")
    primary_info_E = primary_info_E_0.find_element(By.CSS_SELECTOR, 'div[class="col-sm-24"]')
    primary_info = primary_info_E.find_elements(By.CSS_SELECTOR,'article[role="listitem"]') #
    
    for P in primary_info : # itorates through the results
        
        if no_of_results == len(result_list):
            break
        
        business_E_found = "no"
        while business_E_found == "no":
            try:
                sleep(0.02)    
                business_E = P.find_element(By.CSS_SELECTOR, 'a[class="businessCapsule--title"]')
                business = business_E.text # business name
                end_of_the_page = driver.find_element(By.CSS_SELECTOR, 'footer[class="footer"]')#insures that the end of the page is loaded
                business_E_found = "yes"
            except:
                business_E_found = "no"
        business_profile = business_E.get_attribute("href") # the business's profile on yell pages
        
        try: 
            address = P.find_element(By.CSS_SELECTOR, 'span[itemprop="address"]').text # gets the business address
        except:
            address ="not available"
            
            
        try:
            website_E = P.find_element(By.CSS_SELECTOR,'a[data-tracking="WL:CLOSED"]')
            href = website_E.get_attribute("href")
            site = href[href.index('://')+3:]
            if '/' in site :
                website = href[:href.index('://')+3] + site[:site.index("/")+1]
            else:
                website = href[:href.index('://')+3] + site
        except:
            website ="not available"
            
                
        try:
            catagories_E = P.find_element(By.CSS_SELECTOR, 'div[class="col-sm-17 col-md-16 col-lg-18 businessCapsule--classStrap"]')
            catagories = catagories_E.text
        except:
            catagories = "Not Available" 
        
        phone_E = P.find_element(By.CSS_SELECTOR, 'span[class="icon icon-phone business--telephoneIcon"]')
        phone_E.click()
        phone_E2 = P.find_element(By.CSS_SELECTOR, 'div[class="expand--content trans-opacity expand--blockContent business--multiplePhonesWrapper"]')
        phone_3 = phone_E2.find_element(By.CSS_SELECTOR, 'div[class="business--telephoneContent"]')
        phone =phone_3.text.strip().split("\n")[1]       
    
        result = {"Business Name": business, "Profile": business_profile, "Phone Number" :phone,
                "Catagories": catagories, "Address": address, "Website": website}    
        result_list.append(result)
       
    if no_of_results > len(result_list):
        driver.switch_to.default_content()
        ActionChains(driver).send_keys(Keys.END).perform()
       
        ActionChains(driver).send_keys(Keys.END).perform()
        next_page = driver.find_element(By.CSS_SELECTOR,'a[data-tracking="DISPLAY:PAGINATION:NEXT"]')
        next_page.click()
        driver.switch_to.default_content()
        page += 1
        print(f"went to next page{page}")
    
    
        
df_combined = pd.DataFrame(result_list)
driver.quit()

T = str((time.time() - start_time)/60)
minutes = T[:T.index(".")]
seconds = str(int(T[T.index(".")+1:])*60)[:2]
Time =  minutes+"m"+seconds +"s."

print(f"done!!\n{len(result_list)} results scraped successfully in:",Time)

london
10 Pages available.

went to next page2
done!!
50 results scraped successfully in: 1m20s.


In [4]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Business Name  50 non-null     object
 1   Profile        50 non-null     object
 2   Phone Number   50 non-null     object
 3   Catagories     50 non-null     object
 4   Address        50 non-null     object
 5   Website        50 non-null     object
 6   email          50 non-null     object
dtypes: object(7)
memory usage: 2.9+ KB


In [5]:
df_combined.head(5)

,Business Name,Profile,Phone Number,Catagories,Address,Website,email
0,Adara\nSponsored,https://www.yell.com/biz/adara-london-10374362/,020 4579 2698,Architects,We serve London,https://adara.co.uk/,not available
1,Freedom Homes Architects\nSponsored,https://www.yell.com/biz/freedom-homes-archite...,020 3907 3737,Architects Voted one of the top 3 Architect fi...,We serve London,https://www.freedomhomesarchitects.co.uk/,not available
2,TOCA\nSponsored,https://www.yell.com/biz/toca-london-4622238/,020 3953 9062,Architects '' Excellence by design '',"1a Gatward Close, London, N21 1AS",http://www.toca-uk.com/,not available
3,Adara\nSponsored,https://www.yell.com/biz/adara-london-10374362/,020 4579 2698,Architects,We serve London,https://adara.co.uk/,not available
4,Sustainable Design Consultancy Ltd\nSponsored,https://www.yell.com/biz/sustainable-design-co...,07501 071 877,Architects High Quality Architecture,"24 Cornwall Gardens, London, NW10 2QX",http://www.sustainabledesignconsultancy.co.uk/,not available


> ### Saving The Output file as csv.

In [11]:
df_combined.to_csv(f"Yell {search_subject} in {search_location} {page} pages .csv", index= False)

> ### Gettign The e-mail Of Each Business.

In [9]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Business Name  50 non-null     object
 1   Profile        50 non-null     object
 2   Phone Number   50 non-null     object
 3   Catagories     50 non-null     object
 4   Address        50 non-null     object
 5   Website        50 non-null     object
 6   email          50 non-null     object
dtypes: object(7)
memory usage: 2.9+ KB


In [8]:
# # reading an already saved file
# df_combined = pd.read_csv("Yell architects in london 3 pages .csv")

In [12]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Business Name  50 non-null     object
 1   Profile        50 non-null     object
 2   Phone Number   50 non-null     object
 3   Catagories     50 non-null     object
 4   Address        50 non-null     object
 5   Website        50 non-null     object
 6   email          50 non-null     object
dtypes: object(7)
memory usage: 2.9+ KB


In [10]:
df_combined.head(5)

,Business Name,Profile,Phone Number,Catagorys,Adress,Website,email
0,Spaces Architecture\nSponsored,https://www.yell.com/biz/spaces-architecture-h...,01708 452847,Architects Planning / building control applica...,"We serve London | Hornchurch, RM11",http://www.spacesarchitecture.co.uk/,not available
1,Extension Architecture London\nSponsored,https://www.yell.com/biz/extension-architectur...,020 3409 4215,Architects Architecture & planning under one r...,"Unit 3 River Reach Business Park, 1, Gartons W...",http://www.extensionarchitecture.co.uk/,not available
2,Richard Sneesby Architects\nSponsored,https://www.yell.com/biz/richard-sneesby-archi...,020 3907 3701,Architects,"We serve London | Edgware, HA8",http://www.richardsneesbyarchitects.co.uk/,not available
3,Adara\nSponsored,https://www.yell.com/biz/adara-london-10374362/,020 4579 2698,Architects,We serve London,https://adara.co.uk/,not available
4,Andreas Christodoulou Architectural Associates...,https://www.yell.com/biz/andreas-christodoulou...,020 7459 4359,Architects Award Winning Practice and Service,"42 Granville Place High Rd, London, N12 0AX",https://www.ac-aa.co.uk/,not available


In [11]:
# from Functions import *
# for idx, row in df_combined.iterrows():
#     try:
#         if df_combined.loc[idx,'email'] == "not available" and df_combined.loc[idx,'Website'] != "not available":
#             df_combined.loc[idx,'email'] = get_mail(df_combined.loc[idx,'Website'])
#     except:
#         df_combined.loc[idx,'email'] = "Not Available"
#         print("website problem!!")

In [13]:
# df_combined.to_excel(f"Yell {search_subject} in {search_location} {page} pages 3.xlsx", index= False)